In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, collect_set, format_number, max
import pandas as pd

spark = SparkSession.builder\
  .master("local")\
  .appName("Pyspark_SQL")\
  .config("spark.ui.port", '4050')\
  .getOrCreate()
df = spark.read.option("Header", True).csv("/content/Air_Traffic_Passenger_Statistics.csv")

Cuantas companias diferentes aparecen en el fichero

In [ ]:
df.select('Operating Airline').distinct().show()
number_airlines = df.select('Operating Airline').distinct().count()

print("En el registro hay {} diferentes companias".format(number_airlines))

+--------------------+
|   Operating Airline|
+--------------------+
|          Icelandair|
|         Ameriflight|
|      Cathay Pacific|
|          Aeromexico|
|      Etihad Airways|
| Philippine Airlines|
|United Airlines -...|
|    Turkish Airlines|
| Swiss International|
|    Independence Air|
|Miami Air Interna...|
|          Air France|
|      Japan Airlines|
|    Midwest Airlines|
|      Atlas Air, Inc|
|    JetBlue Airways |
|       China Eastern|
|   Mexicana Airlines|
|         Air Canada |
|       Allegiant Air|
+--------------------+
only showing top 20 rows

En el registro hay 77 diferentes companias


Pasajeros de media en cada compania

In [ ]:
df = df.withColumn("Passenger Count", col("Passenger Count").cast("integer"))
result = df.groupBy("Operating Airline").avg("Passenger Count")
result = result.withColumn("avg(Passenger Count)", format_number(col("avg(Passenger Count)"), 2))
result.show()

+--------------------+--------------------+
|   Operating Airline|avg(Passenger Count)|
+--------------------+--------------------+
|          Icelandair|            2,799.70|
|         Ameriflight|                5.00|
|      Cathay Pacific|           17,121.33|
|          Aeromexico|            5,463.82|
|      Etihad Airways|            6,476.09|
| Philippine Airlines|           10,248.64|
|United Airlines -...|           48,915.47|
|    Turkish Airlines|            8,162.42|
| Swiss International|            6,061.64|
|    Independence Air|            6,391.30|
|Miami Air Interna...|              107.38|
|          Air France|           11,589.08|
|      Japan Airlines|            6,470.33|
|    Midwest Airlines|            3,883.00|
|      Atlas Air, Inc|               34.00|
|    JetBlue Airways |           35,261.14|
|       China Eastern|            5,498.40|
|   Mexicana Airlines|            7,993.81|
|         Air Canada |           18,251.56|
|       Allegiant Air|          

Eliminaremos los registros duplicados por el campo “GEO Región”,manteniendo
únicamente aquel con mayor número de pasajeros.

In [ ]:
df = df.withColumn("Passenger Count", col("Passenger Count").cast("integer"))
deduplicated_df = df.groupBy("GEO Region").agg(max("Passenger Count")\
                                               .alias("Max Passenger Count"))

deduplicated_df.show()

+-------------------+-------------------+
|         GEO Region|Max Passenger Count|
+-------------------+-------------------+
|             Europe|              48136|
|    Central America|               8970|
|                 US|             659837|
|      South America|               3685|
|             Mexico|              29206|
|        Middle East|              14769|
|             Canada|              39798|
|Australia / Oceania|              12973|
|               Asia|              86398|
+-------------------+-------------------+



Volcaremos los resultados de los dos puntos anteriores a un CSV

In [ ]:
result.write.mode("overwrite").csv("/content/sample_data/passenger_average.csv", header=True)
deduplicated_df.write.mode("overwrite").csv("/content/single_registers.csv", header=True)